In [2]:
# Needed for one hot encoding
def convert_to_categories(n):
    if n == 0 :
        return 0
    if n == 5:
        return 1
    if n == 10:
        return 2
    
def convert_to_binary(n):
    if n == 0:
        return 0
    else:
        return 1

In [3]:
import csv
import os
import numpy as np

TRAINING_DATA_PATH='/home/mlsg/DiskA/others/ratios'
VALIDATION_DATA_PATH='/home/mlsg/DiskB/others/ratios'
SEQUENCE_LENGTH = 240
TIME_LENGTH = 5
conversion = convert_to_binary

# CSV files format
# [frame, eye aspect ratio, eye_circularity, mouth aspect ratio, mar/ear, 4 more(normalized versions of left 4 in order)]

# Should return an array
def get_row(row):
    return row[5:9] 

label_0 = []
label_5 = []
label_10 = []

# For now we only care about eye aspect ratio
for csv_dir in os.listdir(TRAINING_DATA_PATH):
# # TODO: Change to load more data
# for csv_dir in ['12']:
    full_path = TRAINING_DATA_PATH + '/' + csv_dir
    endings = ['0', '5', '10']
    
    for ending in endings:
        rows = []
        with open(full_path + '/' + ending + '.csv') as f:
            reader = csv.reader(f)
            for row in reader:
                if len(row) <= 5:
                    break
                rows.append(row)
#         Sort by frame number
        rows.sort(key=lambda x:int(x[0]))
        rows = [get_row(row) for row in rows]
        
        # Too few faces, bad video
        if len(rows) < SEQUENCE_LENGTH:
            continue
        # Get the last few rows
        freqs = rows[-SEQUENCE_LENGTH:]
        
        if ending == '0':
            label_0 += (freqs)
        elif ending == '5':
            kkk = 2
#             label_5 += (freqs)
        else:
            label_10 += (freqs)


In [4]:
# Generate the validation data
validation_label_0 = []
validation_label_5 = []
validation_label_10 = []
# For now we only care about eye aspect ratio
for csv_dir in os.listdir(VALIDATION_DATA_PATH):
# # TODO: Change to load more data
# for csv_dir in ['12']:
    full_path = VALIDATION_DATA_PATH + '/' + csv_dir
    endings = ['0', '5', '10']
    
    for ending in endings:
        rows = []
        with open(full_path + '/' + ending + '.csv') as f:
            reader = csv.reader(f)
            for row in reader:
                if len(row) <= 5:
                    break
                rows.append(row)
        # Sort by frame number
        rows.sort(key=lambda x:int(x[0]))
        rows = [get_row(row) for row in rows]
        
        # Too few faces, bad video
        if len(rows) < SEQUENCE_LENGTH:
            continue
        # Get the last few rows from video
        freqs = rows[-SEQUENCE_LENGTH:]

        if ending == '0':
            validation_label_0 += (freqs)
        elif ending== '5':
            kkk = 2
            # do nothing
#             validation_label_5 += (freqs)
        else:
            validation_label_10 += (freqs)

x_validation_data = validation_label_0 + validation_label_5 + validation_label_10
y_validation_data = [0] * len(validation_label_0) + [5] * len(validation_label_5) + [10] * len(validation_label_10)

new_valid_data_y = []
# Label vaidation data
for i in range(0, len(y_validation_data), TIME_LENGTH):
    new_valid_data_y.append(conversion(y_validation_data[i]))
y_validation_data = np.array(new_valid_data_y)

len(x_validation_data)


9600

In [5]:
data_x = label_0 + label_5 + label_10
data_x = np.array(data_x)

# Get them labels
data_y = [0] * len(label_0) + [5] * len(label_5) + [10] * len(label_10)
# Label training data
new_data_y = []
for i in range(0, len(data_y), TIME_LENGTH):
    new_data_y.append(conversion(data_y[i]))
data_y = np.array(new_data_y)

# data_x = np.array(data_x)
input_length = len(data_y)
data_x.shape


(15600, 4)

In [6]:
import random

data_x = data_x.reshape(len(data_x) // TIME_LENGTH, TIME_LENGTH, 4)
x_validation_data = np.array(x_validation_data).reshape(len(x_validation_data) // TIME_LENGTH, TIME_LENGTH, 4)

# c = list(zip(data_x, data_y))
# random.shuffle(c)
# data_x, data_y = zip(*c)

# d = list(zip(x_validation_data, y_validation_data))
# random.shuffle(d)
# x_validation_data, y_validation_data = zip(*d)

data_x = np.array(data_x)
data_y = np.array(data_y)
x_validation_data = np.array(x_validation_data)
y_validation_data = np.array(y_validation_data)

In [68]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding, Flatten, Bidirectional
from keras.optimizers import Adam, RMSprop, Nadam, SGD

model = Sequential()

model.add(Dense(SEQUENCE_LENGTH, activation='sigmoid'))
model.add((LSTM(4, return_sequences=True,
                       input_shape=(TIME_LENGTH, 4),
                       dropout=0.5)))

model.add(Flatten())
model.add(Dense(256, activation='sigmoid')) #FC1
model.add(Dropout(0.5))
model.add(Dense(8, activation='relu')) #FC2
model.add(Dropout(0.5))
model.add(Dense(1, activation='tanh'))#Output Layer
optimizer = Nadam()
# Compile the model
model.compile(
    optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


summary = model.fit(data_x, data_y, epochs=30, validation_data=(x_validation_data, y_validation_data), batch_size=10)
# summary = model.fit(data_x, data_y, epochs=2, validation_split=0.5, batch_size=10)

model.summary()

Train on 3120 samples, validate on 1920 samples
Epoch 1/30
3120/3120 [==============================] - 5s 2ms/step - loss: 4.3700 - accuracy: 0.2949 - val_loss: 0.7478 - val_accuracy: 0.5000
Epoch 2/30
3120/3120 [==============================] - 3s 889us/step - loss: 0.9784 - accuracy: 0.5212 - val_loss: 0.8001 - val_accuracy: 0.5000
Epoch 3/30
3120/3120 [==============================] - 3s 896us/step - loss: 0.7994 - accuracy: 0.5756 - val_loss: 0.6337 - val_accuracy: 0.6411
Epoch 4/30
3120/3120 [==============================] - 3s 873us/step - loss: 0.7238 - accuracy: 0.6244 - val_loss: 0.6115 - val_accuracy: 0.6714
Epoch 5/30
3120/3120 [==============================] - 3s 877us/step - loss: 0.6820 - accuracy: 0.6356 - val_loss: 0.5839 - val_accuracy: 0.6906
Epoch 6/30
3120/3120 [==============================] - 3s 899us/step - loss: 0.6606 - accuracy: 0.6446 - val_loss: 0.5950 - val_accuracy: 0.6818
Epoch 7/30
3120/3120 [==============================] - 3s 877us/step - loss: 

In [ ]:
# Findings: Optimal seqence length at 15 (not 30), optional dropout >= 0.5, need the relu activation
# Circularity of eye is not particularly useful

# DO NOT TOUCH

In [20]:
# DO NOT TOUCH

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding, Flatten, Bidirectional
from keras.optimizers import Adam, RMSprop, SGD
from keras.utils import to_categorical

# DO NOT TOUCH, # DO NOT TOUCH

# Alternative optimize
sgd = SGD(lr=0.01, momentum=0.9)

model = Sequential()

# Use to_categorical to convert to one hot encoding
encoded_data_y = to_categorical(data_y)
encoded_y_validation_data = to_categorical(y_validation_data)

# model.add(Dense(SEQUENCE_LENGTH, activation='sigmoid'))
model.add((LSTM(4, return_sequences=True,
                       input_shape=(TIME_LENGTH, 4,),
                       dropout=0.7)))

model.add(Flatten())
# Dense is fully connected layer. 16 hidden units
# activation for lstm is basically sigmoid or tanh
model.add(Dense(4, activation='relu')) #FC1
model.add(Dropout(0.7))
model.add(Dense(3, activation='softmax'))#Output Layer
optimizer = 'adam'
# Compile the model
model.compile(
    optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


summary = model.fit(data_x, encoded_data_y, epochs=40, validation_data=(x_validation_data, encoded_y_validation_data), batch_size=20)
# summary = model.fit(data_x, data_y, epochs=2, validation_split=0.5, batch_size=10)

model.summary()

Train on 3120 samples, validate on 1920 samples
Epoch 1/40
3120/3120 [==============================] - 2s 722us/step - loss: 1.1162 - accuracy: 0.4769 - val_loss: 0.9666 - val_accuracy: 0.6479
Epoch 2/40
3120/3120 [==============================] - 1s 379us/step - loss: 0.9809 - accuracy: 0.5378 - val_loss: 0.9078 - val_accuracy: 0.6828
Epoch 3/40
3120/3120 [==============================] - 1s 387us/step - loss: 0.9208 - accuracy: 0.5462 - val_loss: 0.8658 - val_accuracy: 0.6755
Epoch 4/40
3120/3120 [==============================] - 1s 386us/step - loss: 0.8782 - accuracy: 0.5356 - val_loss: 0.8306 - val_accuracy: 0.6625
Epoch 5/40
3120/3120 [==============================] - 1s 389us/step - loss: 0.8427 - accuracy: 0.5426 - val_loss: 0.8039 - val_accuracy: 0.6578
Epoch 6/40
3120/3120 [==============================] - 1s 376us/step - loss: 0.8188 - accuracy: 0.5391 - val_loss: 0.7844 - val_accuracy: 0.6594
Epoch 7/40
3120/3120 [==============================] - 1s 374us/step - loss

In [ ]:
# Joshua notes
"""
Switching to categorical cross entropy drastically increased performance
My guess is that this is because we have 3 categories in the first place, so we had to use some form of hot encoding 
to get the correct answer

My first try was only using mouth aspect ratio and mar/ear, both for the normalized and unnormaized values

My second try switched to all the normalized ratios, and it improved things even more

Ths above was done with dropout 0.6 and LSTM number 8, with adam optimizer

Increasing number of neurons also drastically overfits which is a problem

Best so far, 8, 4, 4, 3 around 50% accuracy

Notes and TODO: try varying the 5 to 15 and more for the sequences, maybe feed in more frames also.

dropout 0.7, 4,3,3 with 240, 5 achieved decent results of mid 60s to low 70s with 20 epochs, past that overfitting destroys everythng
"""


In [ ]:
# Sean
""""
Changing optimizer to Nadam with default parameters seems to yield pretty good result 70-75%. dropout rate=0.5. Higher rate decreases accuracy.

Increasing second layer units decreased accuracy drastically. 16 <10%

Increasing epochs and batch size has seemingly no effect.

Rate = 0.5 seems optimal.
""""


In [30]:
# # Confidence interval
# import keras.backend as K
# f = K.function([model.layers[0].input, K.learning_phase()],
#                [model.layers[-1].output])

# def predict_with_uncertainty(f, x, n_iter=10):
#     result = []

#     for i in range(n_iter):
#         result.append(f([x, 1]))

#     result = np.array(result)

#     prediction = result.mean(axis=0)
#     uncertainty = result.var(axis=0)
#     return prediction, uncertainty

# prediction, uncertainty = predict_with_uncertainty(f, x_validation_data)

In [52]:
def new_model():
    model = Sequential()

    model.add(Dense(SEQUENCE_LENGTH, activation='sigmoid'))
    model.add((LSTM(4, return_sequences=True,
                           input_shape=(TIME_LENGTH, 4),
                           dropout=0.5)))

    model.add(Flatten())
    model.add(Dense(256, activation='sigmoid')) #FC1
    model.add(Dropout(0.5))
    model.add(Dense(8, activation='relu')) #FC2
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='tanh'))#Output Layer
    optimizer = Nadam()
    # Compile the model
    model.compile(
        optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [55]:
def train_and_evaluate(model, train_x, train_y, test_x, test_y):
    model.fit(train_x, train_y, epochs=30, batch_size=10)
    # evaluate the model
    scores = model.evaluate(test_x, test_y, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    return scores

In [56]:
from sklearn.cross_validation import StratifiedKFold

labels = np.concatenate([data_y, y_validation_data])
data = np.concatenate([data_x, x_validation_data])
# labels = data_y.concat(data_x)
scores = []
n_folds = 5
    
skf = StratifiedKFold(labels, n_folds=n_folds, shuffle=True)
for i, (train, test) in enumerate(skf):
        eval_model = None # Clearing the NN.
        eval_model = new_model()
        score = train_and_evaluate(eval_model, data[train], labels[train], data[test], labels[test])
        scores.append(score[1])

Epoch 1/30
4031/4031 [==============================] - 4s 895us/step - loss: 1.0628 - accuracy: 0.5125
Epoch 2/30
4031/4031 [==============================] - 2s 542us/step - loss: 0.7019 - accuracy: 0.6197
Epoch 3/30
4031/4031 [==============================] - 2s 549us/step - loss: 0.6350 - accuracy: 0.6634
Epoch 4/30
4031/4031 [==============================] - 2s 540us/step - loss: 0.6163 - accuracy: 0.6653
Epoch 5/30
4031/4031 [==============================] - 2s 550us/step - loss: 0.6102 - accuracy: 0.6683
Epoch 6/30
4031/4031 [==============================] - 2s 549us/step - loss: 0.6050 - accuracy: 0.6765
Epoch 7/30
4031/4031 [==============================] - 2s 551us/step - loss: 0.6092 - accuracy: 0.6706
Epoch 8/30
4031/4031 [==============================] - 2s 546us/step - loss: 0.6008 - accuracy: 0.6820
Epoch 9/30
4031/4031 [==============================] - 2s 543us/step - loss: 0.5996 - accuracy: 0.6785
Epoch 10/30
4031/4031 [==============================] - 2s 546u

Epoch 19/30
4032/4032 [==============================] - 2s 545us/step - loss: 0.5972 - accuracy: 0.6882
Epoch 20/30
4032/4032 [==============================] - 2s 552us/step - loss: 0.6004 - accuracy: 0.6776
Epoch 21/30
4032/4032 [==============================] - 2s 546us/step - loss: 0.5918 - accuracy: 0.6902
Epoch 22/30
4032/4032 [==============================] - 2s 540us/step - loss: 0.5914 - accuracy: 0.6900
Epoch 23/30
4032/4032 [==============================] - 2s 547us/step - loss: 0.5843 - accuracy: 0.6964
Epoch 24/30
4032/4032 [==============================] - 2s 546us/step - loss: 0.5841 - accuracy: 0.6910
Epoch 25/30
4032/4032 [==============================] - 2s 539us/step - loss: 0.5839 - accuracy: 0.6922
Epoch 26/30
4032/4032 [==============================] - 2s 552us/step - loss: 0.5874 - accuracy: 0.6947
Epoch 27/30
4032/4032 [==============================] - 2s 549us/step - loss: 0.5829 - accuracy: 0.6947
Epoch 28/30
4032/4032 [==============================] 

In [57]:
# ROC score
pred = model.predict(x_validation_data).ravel()

In [62]:
from sklearn.metrics import roc_curve
from sklearn import metrics

fpr, tpr, thresholds = roc_curve(y_validation_data, pred)
roc_auc = metrics.auc(fpr, tpr)
roc_auc

0.7322721354166666

In [66]:
tpr

array([0.00104167, 0.17291667, 0.17291667, 0.17395833, 0.17395833,
       0.190625  , 0.190625  , 0.20208333, 0.20208333, 0.22916667,
       0.22916667, 0.23645833, 0.23645833, 0.24166667, 0.24166667,
       0.24270833, 0.24270833, 0.25208333, 0.25208333, 0.25520833,
       0.25520833, 0.25729167, 0.25729167, 0.26041667, 0.26041667,
       0.26458333, 0.26458333, 0.26875   , 0.26875   , 0.27708333,
       0.27708333, 0.28020833, 0.28020833, 0.2875    , 0.2875    ,
       0.30208333, 0.30208333, 0.30416667, 0.30416667, 0.30520833,
       0.30520833, 0.30729167, 0.30729167, 0.30833333, 0.30833333,
       0.309375  , 0.309375  , 0.31666667, 0.31666667, 0.32291667,
       0.32291667, 0.32395833, 0.32395833, 0.32604167, 0.32604167,
       0.33125   , 0.33125   , 0.33541667, 0.33541667, 0.3375    ,
       0.3375    , 0.340625  , 0.340625  , 0.34583333, 0.34583333,
       0.346875  , 0.346875  , 0.34895833, 0.34895833, 0.353125  ,
       0.353125  , 0.35520833, 0.35520833, 0.36041667, 0.36041